In [1]:
import numpy as np
import pandas as pd
import datetime
import time
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

%run ./../custom_functions/train_dev_test_split.ipynb
%run ./../custom_functions/get_future_predictions.ipynb

from sklearn.ensemble import RandomForestRegressor

In [2]:
marag_df = pd.read_csv('./../../../Databases/clean_data/marag_lags_data.csv')
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)

marag_daily = pd.read_csv('./../../../Databases/clean_data/marag_dailylags_data.csv')
marag_daily.set_index('Timestamp', inplace=True)
marag_daily.sort_index(inplace=True)

# For the time being we will not use the outliers column, so we will drop it
marag_df.drop(['outliers'], axis=1, inplace=True)

In [3]:
# Let's import the marag data with no lags for future comparative purposes
no_lags_marag = pd.read_csv('./../../../Databases/clean_data/marag_data.csv')
no_lags_marag.set_index('Timestamp', inplace=True)
no_lags_marag.sort_index(inplace=True)
no_lags_marag.drop(['outliers'], axis=1, inplace=True)

daily_nolags_marag = pd.read_csv('./../../../Databases/clean_data/marag_daily_data.csv')
daily_nolags_marag.set_index('Timestamp', inplace=True)
daily_nolags_marag.sort_index(inplace=True)

In [78]:
# Let's create a dataframe in which we'll append all the estimated predictions, and another
# containing the running times of each scenario.

rf_results_df = pd.DataFrame
rf_runtime_df = pd.DataFrame

rf_10_df = pd.DataFrame
rf_runtime10 = pd.DataFrame

rf_dailyresults_df = pd.DataFrame
rf_runtime_dailydf = pd.DataFrame

## Model tuning

In [6]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])
rftune_X_train = pd.concat([X_train, X_dev])
rftune_X_train.sort_index(inplace=True)
rftune_y_train = pd.concat([y_train, y_dev])
rftune_y_train.sort_index(inplace=True)

In [8]:
# RandomForest baseline

rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)
RF_baseline_y_hat = rf.predict(X_dev)

/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
baseline_RMSE = pd.DataFrame(pd.Series(np.sqrt(mean_squared_error(y_dev, RF_baseline_y_hat)), name = 'RMSE'))
baseline_RMSE

,RMSE
0,33.85374


In [13]:
n_estimators = [100, 200, 400, 600]
max_depth = [10, 30, 60, 100]
best_RMSE = []

start_rf_tune = time.time()

for ne in tqdm(n_estimators):
    for md in tqdm(max_depth):
        rf = RandomForestRegressor(n_estimators = ne, max_depth = md, n_jobs = -1, random_state = 42)
        rf.fit(X_train, y_train)
        y_dev_hat = rf.predict(X_dev)
        RMSE_rfparams = sqrt(mean_squared_error(y_dev, y_dev_hat))
        
        if pd.DataFrame(best_RMSE).empty:
            best_RMSE = [ne, md, RMSE_rfparams]
        else:
            if RMSE_rfparams < best_RMSE[2]:
                best_RMSE = [ne, md, RMSE_rfparams]
        
end_rf_tune = time.time()
rf_tune_runtime = end_rf_tune - start_rf_tune



  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [01:25<04:17, 85.93s/it]


 50%|█████     | 2/4 [03:34<03:17, 98.70s/it]


 75%|███████▌  | 3/4 [05:45<01:48, 108.45s/it]


100%|██████████| 4/4 [07:58<00:00, 115.83s/it]

 25%|██▌       | 1/4 [07:58<23:56, 478.67s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [02:45<08:16, 165.62s/it]


 50%|█████     | 2/4 [07:01<06:25, 192.77s/it]


 75%|███████▌  | 3/4 [11:17<03:31, 211.67s/it]


100%|██████████| 4/4 [15:35<00:00, 225.67s/it]

 50%|█████     | 2/4 [23:34<20:31, 615.83s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [05:20<16:00, 320.09s/it]


 50%|█████     | 2/4 [13:52<12:35, 377.70s/it]


 75%|███████▌  | 3/4 [22:35<07:01, 421.41s/it]


100%|██████████| 4/4 [31:05<00:00, 447.97s/it]

 75%|███████▌  | 3/4 [54:40<16:30, 990.75s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [07:57<23:52, 477.61s/it]


 50%|█████     | 2/

In [14]:
print(best_RMSE, rf_tune_runtime)

[400, 30, 32.32575519464866] 6071.081792593002


The above tuning method is a simplification of the one intended to be used initially. The combination of hyperparameters implied the running of more than 4000 iterations and it was not a feasible achievement for the scope of this project (some of those iterations took up to 95 minutes to run).

## Original data (48 obsv. / day)

### All columns, known future

Future information will be used as the predictions will be forecast using lags of future observations.

In [80]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

rf_X_train = pd.concat([X_train, X_dev])
rf_X_train.sort_index(inplace=True)
rf_y_train = pd.concat([y_train, y_dev])
rf_y_train.sort_index(inplace=True)

In [82]:
start_PKF_allcols = time.time()

rf_PKF = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, verbose=10, random_state=42)
rf_PKF.fit(rf_X_train, rf_y_train)
rf_y_hat = rf_PKF.predict(X_test)
rf_y_hat = pd.Series(rf_y_hat, index=y_test.index, name='TotEntr_PKF_allcols')

end_PKF_allcols = time.time()
rf_runtime_PKF_allcols = end_PKF_allcols - start_PKF_allcols
rf_runtime_PKF_allcols = pd.DataFrame(pd.Series(rf_runtime_PKF_allcols, name = 'PKF_allcols_rf_runtime'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400building tree 4 of 400
building tree 5 of 400

building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.4s


building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.8s


...

[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  5.4min


building tree 357 of 400
building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400


[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:  5.8min


building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  6.2min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    

In [83]:
rf_observed_values = pd.DataFrame(y_test)
rf_observed_values.columns = ['Observed_values']
rf_results_df = rf_observed_values.merge(pd.DataFrame(rf_y_hat), how='left', on='Timestamp')
rf_results_df.head()

,Observed_values,TotEntr_PKF_allcols
Timestamp,,
2018-09-24 00:00:00,20,15.470000
2018-09-24 00:30:00,0,0.772719
2018-09-24 01:00:00,0,0.000000
2018-09-24 01:30:00,0,0.000000
2018-09-24 02:00:00,0,0.000000


In [88]:
rf_runtime_df = rf_runtime_PKF_allcols
rf_runtime_df.head()

,PKF_allcols_rf_runtime
0,371.837343


### Just lags, known future

Using just lags of future observations.

In [89]:
lags_PKF_df = marag_df.drop(['Temperature', 'Precipitation', 'Open/Closed', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, lags_X_test, y_train, y_dev, lags_y_test = train_dev_test_split(lags_PKF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_PKF_df['TotalEntries'])

rf_lags_X_train = pd.concat([X_train, X_dev])
rf_lags_X_train.sort_index(inplace=True)
rf_lags_y_train = pd.concat([y_train, y_dev])
rf_lags_y_train.sort_index(inplace=True)

In [90]:
start_PKF_lags = time.time()
rf_lags_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, verbose=10, random_state=42)
rf_lags_model.fit(rf_lags_X_train, rf_lags_y_train)
rf_lags_y_hat = rf_lags_model.predict(lags_X_test)
rf_lags_y_hat = pd.Series(rf_lags_y_hat, index=lags_y_test.index, name = 'TotEntr_PKF_lags')

end_PKF_lags = time.time()
rf_runtime_PKF_lags = end_PKF_lags - start_PKF_lags
rf_runtime_PKF_lags = pd.DataFrame(pd.Series(rf_runtime_PKF_lags, name = 'PKF_lags_rf_runtime'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 400building tree 2 of 400
building tree 3 of 400

building tree 4 of 400building tree 5 of 400building tree 6 of 400

building tree 7 of 400

building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.5s


building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.7s


...

[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  5.6min


building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400


[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:  6.1min


building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  6.4min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    

In [91]:
rf_results_df = rf_results_df.merge(rf_lags_y_hat, on='Timestamp')
rf_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags
Timestamp,,,
2018-09-24 00:00:00,20,15.470000,15.475000
2018-09-24 00:30:00,0,0.772719,0.703395
2018-09-24 01:00:00,0,0.000000,0.000000
2018-09-24 01:30:00,0,0.000000,0.000000
2018-09-24 02:00:00,0,0.000000,0.000000


In [94]:
rf_runtime_df = pd.concat([rf_runtime_df, rf_runtime_PKF_lags], axis = 1)
rf_runtime_df.head()

,PKF_allcols_rf_runtime,PKF_lags_rf_runtime
0,371.837343,386.499835


When using the get_future_preds function, including the refit of the model with each new prediction takes more than 6 minutes per iteration. Considering the dimensions of our test portion of data (with 7392 rows), that would mean a computation time of one month approximately. That's out of the scope of the present project (because of the limited time of development), so 2 scenarios will be explored:

    - Making predictions for the whole X_test without refiting the model each iteration
    - Predicting 10 days of results refiting the model each iteration

### No model refitting: All columns, unknown future

In [4]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

rf_X_train = pd.concat([X_train, X_dev])
rf_X_train.sort_index(inplace=True)
rf_y_train = pd.concat([y_train, y_dev])
rf_y_train.sort_index(inplace=True)

In [5]:
rf_X_train.shape

(29568, 340)

In [ ]:
rf_PUF_allcols_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, random_state=42)
rf_PUF_allcols_model.fit(rf_X_train, rf_y_train)

In [72]:
start_PUF_allcols = time.time()

rf_allcols_df = get_future_preds(no_lags_marag, rf_X_train, X_test, rf_PUF_allcols_model, refit=False)

end_PUF_allcols = time.time()
rf_runtime_PUF_allcols = end_PUF_allcols - start_PUF_allcols
rf_runtime_PUF_allcols = pd.DataFrame(pd.Series(rf_runtime_PUF_allcols, name = 'PUF_allcols_rf_runtime'))
rf_allcols_df.columns = ['TotEntr_PUF_allcols']






  0%|          | 0/7392 [00:00<?, ?it/s]




  0%|          | 1/7392 [00:00<55:30,  2.22it/s]




  0%|          | 2/7392 [00:01<1:05:32,  1.88it/s]




  0%|          | 3/7392 [00:01<1:11:59,  1.71it/s]




  0%|          | 4/7392 [00:02<1:16:59,  1.60it/s]




  0%|          | 5/7392 [00:03<1:20:22,  1.53it/s]




  0%|          | 6/7392 [00:04<1:23:32,  1.47it/s]




  0%|          | 7/7392 [00:04<1:24:40,  1.45it/s]




  0%|          | 8/7392 [00:05<1:25:50,  1.43it/s]




  0%|          | 9/7392 [00:06<1:25:54,  1.43it/s]




  0%|          | 10/7392 [00:06<1:27:27,  1.41it/s]




  0%|          | 11/7392 [00:07<1:27:29,  1.41it/s]




  0%|          | 12/7392 [00:08<1:29:12,  1.38it/s]




  0%|          | 13/7392 [00:09<1:29:53,  1.37it/s]




  0%|          | 14/7392 [00:09<1:29:04,  1.38it/s]




  0%|          | 15/7392 [00:10<1:27:54,  1.40it/s]




  0%|          | 16/7392 [00:11<1:27:40,  1.40it/s]




  0%|          | 17/7392 [00:11<1:26:42,  1.42it/s]




  0%|    

  3%|▎         | 228/7392 [02:43<1:25:00,  1.40it/s]




  3%|▎         | 229/7392 [02:44<1:25:57,  1.39it/s]




  3%|▎         | 230/7392 [02:45<1:25:30,  1.40it/s]




  3%|▎         | 231/7392 [02:46<1:25:08,  1.40it/s]




  3%|▎         | 232/7392 [02:46<1:24:53,  1.41it/s]




  3%|▎         | 233/7392 [02:47<1:24:34,  1.41it/s]




  3%|▎         | 234/7392 [02:48<1:25:07,  1.40it/s]




  3%|▎         | 235/7392 [02:48<1:24:49,  1.41it/s]




  3%|▎         | 236/7392 [02:49<1:25:05,  1.40it/s]




  3%|▎         | 237/7392 [02:50<1:24:24,  1.41it/s]




  3%|▎         | 238/7392 [02:51<1:24:59,  1.40it/s]




  3%|▎         | 239/7392 [02:51<1:24:34,  1.41it/s]




  3%|▎         | 240/7392 [02:52<1:24:21,  1.41it/s]




  3%|▎         | 241/7392 [02:53<1:24:05,  1.42it/s]




  3%|▎         | 242/7392 [02:53<1:24:07,  1.42it/s]




  3%|▎         | 243/7392 [02:54<1:28:20,  1.35it/s]




  3%|▎         | 244/7392 [02:55<1:38:44,  1.21it/s]




  3%|▎         | 245/7392 [02:5

  6%|▌         | 454/7392 [05:24<1:23:02,  1.39it/s]




  6%|▌         | 455/7392 [05:25<1:22:31,  1.40it/s]




  6%|▌         | 456/7392 [05:26<1:26:35,  1.34it/s]




  6%|▌         | 457/7392 [05:26<1:25:22,  1.35it/s]




  6%|▌         | 458/7392 [05:27<1:24:19,  1.37it/s]




  6%|▌         | 459/7392 [05:28<1:23:13,  1.39it/s]




  6%|▌         | 460/7392 [05:28<1:22:46,  1.40it/s]




  6%|▌         | 461/7392 [05:29<1:22:05,  1.41it/s]




  6%|▋         | 462/7392 [05:30<1:22:14,  1.40it/s]




  6%|▋         | 463/7392 [05:31<1:21:40,  1.41it/s]




  6%|▋         | 464/7392 [05:31<1:22:05,  1.41it/s]




  6%|▋         | 465/7392 [05:32<1:22:55,  1.39it/s]




  6%|▋         | 466/7392 [05:33<1:22:43,  1.40it/s]




  6%|▋         | 467/7392 [05:33<1:22:06,  1.41it/s]




  6%|▋         | 468/7392 [05:34<1:22:23,  1.40it/s]




  6%|▋         | 469/7392 [05:35<1:22:14,  1.40it/s]




  6%|▋         | 470/7392 [05:36<1:22:32,  1.40it/s]




  6%|▋         | 471/7392 [05:3

  9%|▉         | 680/7392 [08:07<1:20:27,  1.39it/s]




  9%|▉         | 681/7392 [08:08<1:20:27,  1.39it/s]




  9%|▉         | 682/7392 [08:08<1:19:54,  1.40it/s]




  9%|▉         | 683/7392 [08:09<1:19:50,  1.40it/s]




  9%|▉         | 684/7392 [08:10<1:23:06,  1.35it/s]




  9%|▉         | 685/7392 [08:11<1:22:07,  1.36it/s]




  9%|▉         | 686/7392 [08:11<1:21:26,  1.37it/s]




  9%|▉         | 687/7392 [08:12<1:20:56,  1.38it/s]




  9%|▉         | 688/7392 [08:13<1:25:15,  1.31it/s]




  9%|▉         | 689/7392 [08:14<1:23:51,  1.33it/s]




  9%|▉         | 690/7392 [08:14<1:22:55,  1.35it/s]




  9%|▉         | 691/7392 [08:15<1:26:31,  1.29it/s]




  9%|▉         | 692/7392 [08:16<1:25:11,  1.31it/s]




  9%|▉         | 693/7392 [08:17<1:23:45,  1.33it/s]




  9%|▉         | 694/7392 [08:17<1:26:35,  1.29it/s]




  9%|▉         | 695/7392 [08:18<1:24:30,  1.32it/s]




  9%|▉         | 696/7392 [08:19<1:23:10,  1.34it/s]




  9%|▉         | 697/7392 [08:2

 12%|█▏        | 906/7392 [10:51<1:17:26,  1.40it/s]




 12%|█▏        | 907/7392 [10:52<1:17:21,  1.40it/s]




 12%|█▏        | 908/7392 [10:53<1:17:47,  1.39it/s]




 12%|█▏        | 909/7392 [10:53<1:17:46,  1.39it/s]




 12%|█▏        | 910/7392 [10:54<1:17:31,  1.39it/s]




 12%|█▏        | 911/7392 [10:55<1:17:52,  1.39it/s]




 12%|█▏        | 912/7392 [10:55<1:17:31,  1.39it/s]




 12%|█▏        | 913/7392 [10:56<1:17:46,  1.39it/s]




 12%|█▏        | 914/7392 [10:57<1:18:46,  1.37it/s]




 12%|█▏        | 915/7392 [10:58<1:18:10,  1.38it/s]




 12%|█▏        | 916/7392 [10:58<1:17:59,  1.38it/s]




 12%|█▏        | 917/7392 [10:59<1:17:50,  1.39it/s]




 12%|█▏        | 918/7392 [11:00<1:17:39,  1.39it/s]




 12%|█▏        | 919/7392 [11:00<1:17:28,  1.39it/s]




 12%|█▏        | 920/7392 [11:01<1:17:41,  1.39it/s]




 12%|█▏        | 921/7392 [11:02<1:17:38,  1.39it/s]




 12%|█▏        | 922/7392 [11:03<1:17:33,  1.39it/s]




 12%|█▏        | 923/7392 [11:0

...

 86%|████████▌ | 6342/7392 [1:19:11<13:38,  1.28it/s]




 86%|████████▌ | 6343/7392 [1:19:11<13:35,  1.29it/s]




 86%|████████▌ | 6344/7392 [1:19:12<13:32,  1.29it/s]




 86%|████████▌ | 6345/7392 [1:19:13<13:32,  1.29it/s]




 86%|████████▌ | 6346/7392 [1:19:14<13:28,  1.29it/s]




 86%|████████▌ | 6347/7392 [1:19:14<13:35,  1.28it/s]




 86%|████████▌ | 6348/7392 [1:19:15<13:40,  1.27it/s]




 86%|████████▌ | 6349/7392 [1:19:16<13:38,  1.27it/s]




 86%|████████▌ | 6350/7392 [1:19:17<13:28,  1.29it/s]




 86%|████████▌ | 6351/7392 [1:19:18<13:29,  1.29it/s]




 86%|████████▌ | 6352/7392 [1:19:18<13:26,  1.29it/s]




 86%|████████▌ | 6353/7392 [1:19:19<13:24,  1.29it/s]




 86%|████████▌ | 6354/7392 [1:19:20<13:21,  1.30it/s]




 86%|████████▌ | 6355/7392 [1:19:21<13:26,  1.29it/s]




 86%|████████▌ | 6356/7392 [1:19:21<13:26,  1.28it/s]




 86%|████████▌ | 6357/7392 [1:19:22<13:24,  1.29it/s]




 86%|████████▌ | 6358/7392 [1:19:23<14:06,  1.22it/s]




 86%|████████▌

 89%|████████▉ | 6566/7392 [1:22:05<10:44,  1.28it/s]




 89%|████████▉ | 6567/7392 [1:22:06<10:44,  1.28it/s]




 89%|████████▉ | 6568/7392 [1:22:07<10:38,  1.29it/s]




 89%|████████▉ | 6569/7392 [1:22:07<10:39,  1.29it/s]




 89%|████████▉ | 6570/7392 [1:22:08<10:35,  1.29it/s]




 89%|████████▉ | 6571/7392 [1:22:09<10:36,  1.29it/s]




 89%|████████▉ | 6572/7392 [1:22:10<10:39,  1.28it/s]




 89%|████████▉ | 6573/7392 [1:22:10<10:36,  1.29it/s]




 89%|████████▉ | 6574/7392 [1:22:11<10:37,  1.28it/s]




 89%|████████▉ | 6575/7392 [1:22:12<10:34,  1.29it/s]




 89%|████████▉ | 6576/7392 [1:22:13<10:27,  1.30it/s]




 89%|████████▉ | 6577/7392 [1:22:14<10:26,  1.30it/s]




 89%|████████▉ | 6578/7392 [1:22:14<10:29,  1.29it/s]




 89%|████████▉ | 6579/7392 [1:22:15<10:30,  1.29it/s]




 89%|████████▉ | 6580/7392 [1:22:16<10:26,  1.30it/s]




 89%|████████▉ | 6581/7392 [1:22:17<10:27,  1.29it/s]




 89%|████████▉ | 6582/7392 [1:22:17<10:30,  1.29it/s]




 89%|████████▉

 92%|█████████▏| 6790/7392 [1:25:00<07:50,  1.28it/s]




 92%|█████████▏| 6791/7392 [1:25:01<07:47,  1.29it/s]




 92%|█████████▏| 6792/7392 [1:25:02<07:46,  1.29it/s]




 92%|█████████▏| 6793/7392 [1:25:03<07:44,  1.29it/s]




 92%|█████████▏| 6794/7392 [1:25:04<07:42,  1.29it/s]




 92%|█████████▏| 6795/7392 [1:25:04<07:40,  1.30it/s]




 92%|█████████▏| 6796/7392 [1:25:05<07:42,  1.29it/s]




 92%|█████████▏| 6797/7392 [1:25:06<07:42,  1.29it/s]




 92%|█████████▏| 6798/7392 [1:25:07<07:41,  1.29it/s]




 92%|█████████▏| 6799/7392 [1:25:07<07:44,  1.28it/s]




 92%|█████████▏| 6800/7392 [1:25:08<07:40,  1.29it/s]




 92%|█████████▏| 6801/7392 [1:25:09<07:39,  1.29it/s]




 92%|█████████▏| 6802/7392 [1:25:10<07:39,  1.28it/s]




 92%|█████████▏| 6803/7392 [1:25:11<07:39,  1.28it/s]




 92%|█████████▏| 6804/7392 [1:25:11<07:38,  1.28it/s]




 92%|█████████▏| 6805/7392 [1:25:12<07:37,  1.28it/s]




 92%|█████████▏| 6806/7392 [1:25:13<08:20,  1.17it/s]




 92%|█████████

 95%|█████████▍| 7014/7392 [1:27:56<04:56,  1.28it/s]




 95%|█████████▍| 7015/7392 [1:27:57<04:55,  1.28it/s]




 95%|█████████▍| 7016/7392 [1:27:58<04:54,  1.28it/s]




 95%|█████████▍| 7017/7392 [1:27:59<04:55,  1.27it/s]




 95%|█████████▍| 7018/7392 [1:28:00<04:53,  1.27it/s]




 95%|█████████▍| 7019/7392 [1:28:00<04:54,  1.27it/s]




 95%|█████████▍| 7020/7392 [1:28:01<04:53,  1.27it/s]




 95%|█████████▍| 7021/7392 [1:28:02<04:51,  1.27it/s]




 95%|█████████▍| 7022/7392 [1:28:03<04:50,  1.28it/s]




 95%|█████████▌| 7023/7392 [1:28:04<04:48,  1.28it/s]




 95%|█████████▌| 7024/7392 [1:28:04<04:46,  1.28it/s]




 95%|█████████▌| 7025/7392 [1:28:05<04:47,  1.28it/s]




 95%|█████████▌| 7026/7392 [1:28:06<04:45,  1.28it/s]




 95%|█████████▌| 7027/7392 [1:28:07<04:45,  1.28it/s]




 95%|█████████▌| 7028/7392 [1:28:07<04:43,  1.28it/s]




 95%|█████████▌| 7029/7392 [1:28:08<04:43,  1.28it/s]




 95%|█████████▌| 7030/7392 [1:28:09<04:41,  1.29it/s]




 95%|█████████

 98%|█████████▊| 7238/7392 [1:30:53<02:02,  1.26it/s]




 98%|█████████▊| 7239/7392 [1:30:54<02:01,  1.26it/s]




 98%|█████████▊| 7240/7392 [1:30:54<02:00,  1.26it/s]




 98%|█████████▊| 7241/7392 [1:30:55<02:00,  1.25it/s]




 98%|█████████▊| 7242/7392 [1:30:56<01:59,  1.25it/s]




 98%|█████████▊| 7243/7392 [1:30:57<01:58,  1.26it/s]




 98%|█████████▊| 7244/7392 [1:30:58<01:57,  1.26it/s]




 98%|█████████▊| 7245/7392 [1:30:58<01:55,  1.27it/s]




 98%|█████████▊| 7246/7392 [1:30:59<01:54,  1.27it/s]




 98%|█████████▊| 7247/7392 [1:31:00<01:54,  1.27it/s]




 98%|█████████▊| 7248/7392 [1:31:01<01:54,  1.26it/s]




 98%|█████████▊| 7249/7392 [1:31:02<01:54,  1.25it/s]




 98%|█████████▊| 7250/7392 [1:31:03<02:12,  1.08it/s]




 98%|█████████▊| 7251/7392 [1:31:04<02:04,  1.13it/s]




 98%|█████████▊| 7252/7392 [1:31:04<02:00,  1.16it/s]




 98%|█████████▊| 7253/7392 [1:31:05<01:57,  1.19it/s]




 98%|█████████▊| 7254/7392 [1:31:06<01:54,  1.20it/s]




 98%|█████████

In [95]:
rf_results_df = rf_results_df.merge(rf_allcols_df, on='Timestamp')
rf_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags,TotEntr_PUF_allcols
Timestamp,,,,
2018-09-24 00:00:00,20,15.470000,15.475000,15.4700
2018-09-24 00:30:00,0,0.772719,0.703395,55.3450
2018-09-24 01:00:00,0,0.000000,0.000000,38.8900
2018-09-24 01:30:00,0,0.000000,0.000000,32.0025
2018-09-24 02:00:00,0,0.000000,0.000000,15.5425


In [96]:
rf_runtime_df = pd.concat([rf_runtime_df, rf_runtime_PUF_allcols], axis = 1)
rf_runtime_df.head()

,PKF_allcols_rf_runtime,PKF_lags_rf_runtime,PUF_allcols_rf_runtime
0,371.837343,386.499835,5577.384185


### No model refitting: just lags, unknown future

In [98]:
lags_PUF_df = marag_df.drop(['Temperature', 'Precipitation', 'Open/Closed', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, lags_X_test, y_train, y_dev, lags_y_test = train_dev_test_split(lags_PUF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_PUF_df['TotalEntries'])

rf_lags_X_train = pd.concat([X_train, X_dev])
rf_lags_X_train.sort_index(inplace=True)
rf_lags_y_train = pd.concat([y_train, y_dev])
rf_lags_y_train.sort_index(inplace=True)

In [99]:
rf_PUF_lags_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, random_state=42)
rf_PUF_lags_model.fit(rf_lags_X_train, rf_lags_y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [100]:
start_PUF_lags = time.time()

rf_lags_df = get_future_preds(no_lags_marag, rf_lags_X_train, lags_X_test, rf_PUF_lags_model, justlags=True, refit=False)


end_PUF_lags = time.time()
rf_runtime_PUF_lags = end_PUF_lags - start_PUF_lags
rf_runtime_PUF_lags = pd.DataFrame(pd.Series(rf_runtime_PUF_lags, name = 'PUF_lags_rf_runtime'))
rf_lags_df.columns = ['TotEntr_PUF_lags']






  0%|          | 0/7392 [00:00<?, ?it/s]




  0%|          | 1/7392 [00:01<2:17:02,  1.11s/it]




  0%|          | 2/7392 [00:03<2:41:52,  1.31s/it]




  0%|          | 3/7392 [00:03<2:22:28,  1.16s/it]




  0%|          | 4/7392 [00:04<2:04:02,  1.01s/it]




  0%|          | 5/7392 [00:04<1:48:45,  1.13it/s]




  0%|          | 6/7392 [00:05<1:40:50,  1.22it/s]




  0%|          | 7/7392 [00:06<1:35:31,  1.29it/s]




  0%|          | 8/7392 [00:06<1:30:25,  1.36it/s]




  0%|          | 9/7392 [00:07<1:26:26,  1.42it/s]




  0%|          | 10/7392 [00:08<1:23:00,  1.48it/s]




  0%|          | 11/7392 [00:08<1:21:11,  1.52it/s]




  0%|          | 12/7392 [00:09<1:19:58,  1.54it/s]




  0%|          | 13/7392 [00:10<1:18:37,  1.56it/s]




  0%|          | 14/7392 [00:10<1:16:47,  1.60it/s]




  0%|          | 15/7392 [00:11<1:16:02,  1.62it/s]




  0%|          | 16/7392 [00:11<1:15:26,  1.63it/s]




  0%|          | 17/7392 [00:12<1:15:40,  1.62it/s]




  0%|  

  3%|▎         | 228/7392 [02:19<1:13:26,  1.63it/s]




  3%|▎         | 229/7392 [02:20<1:12:54,  1.64it/s]




  3%|▎         | 230/7392 [02:20<1:12:44,  1.64it/s]




  3%|▎         | 231/7392 [02:21<1:13:06,  1.63it/s]




  3%|▎         | 232/7392 [02:21<1:12:31,  1.65it/s]




  3%|▎         | 233/7392 [02:22<1:12:38,  1.64it/s]




  3%|▎         | 234/7392 [02:23<1:13:02,  1.63it/s]




  3%|▎         | 235/7392 [02:23<1:16:34,  1.56it/s]




  3%|▎         | 236/7392 [02:24<1:15:27,  1.58it/s]




  3%|▎         | 237/7392 [02:25<1:14:34,  1.60it/s]




  3%|▎         | 238/7392 [02:25<1:14:01,  1.61it/s]




  3%|▎         | 239/7392 [02:26<1:13:35,  1.62it/s]




  3%|▎         | 240/7392 [02:26<1:13:17,  1.63it/s]




  3%|▎         | 241/7392 [02:27<1:13:10,  1.63it/s]




  3%|▎         | 242/7392 [02:28<1:13:34,  1.62it/s]




  3%|▎         | 243/7392 [02:28<1:14:20,  1.60it/s]




  3%|▎         | 244/7392 [02:29<1:21:04,  1.47it/s]




  3%|▎         | 245/7392 [02:3

  6%|▌         | 454/7392 [04:37<1:09:57,  1.65it/s]




  6%|▌         | 455/7392 [04:38<1:09:45,  1.66it/s]




  6%|▌         | 456/7392 [04:38<1:10:01,  1.65it/s]




  6%|▌         | 457/7392 [04:39<1:10:55,  1.63it/s]




  6%|▌         | 458/7392 [04:39<1:10:42,  1.63it/s]




  6%|▌         | 459/7392 [04:40<1:10:47,  1.63it/s]




  6%|▌         | 460/7392 [04:41<1:12:59,  1.58it/s]




  6%|▌         | 461/7392 [04:41<1:12:35,  1.59it/s]




  6%|▋         | 462/7392 [04:42<1:11:01,  1.63it/s]




  6%|▋         | 463/7392 [04:42<1:10:40,  1.63it/s]




  6%|▋         | 464/7392 [04:43<1:09:54,  1.65it/s]




  6%|▋         | 465/7392 [04:44<1:10:58,  1.63it/s]




  6%|▋         | 466/7392 [04:44<1:09:58,  1.65it/s]




  6%|▋         | 467/7392 [04:45<1:09:24,  1.66it/s]




  6%|▋         | 468/7392 [04:45<1:08:54,  1.67it/s]




  6%|▋         | 469/7392 [04:46<1:08:30,  1.68it/s]




  6%|▋         | 470/7392 [04:47<1:11:20,  1.62it/s]




  6%|▋         | 471/7392 [04:4

  9%|▉         | 680/7392 [06:56<1:08:53,  1.62it/s]




  9%|▉         | 681/7392 [06:56<1:08:00,  1.64it/s]




  9%|▉         | 682/7392 [06:57<1:08:01,  1.64it/s]




  9%|▉         | 683/7392 [06:58<1:07:33,  1.66it/s]




  9%|▉         | 684/7392 [06:58<1:06:57,  1.67it/s]




  9%|▉         | 685/7392 [06:59<1:06:46,  1.67it/s]




  9%|▉         | 686/7392 [06:59<1:07:14,  1.66it/s]




  9%|▉         | 687/7392 [07:00<1:07:30,  1.66it/s]




  9%|▉         | 688/7392 [07:01<1:07:33,  1.65it/s]




  9%|▉         | 689/7392 [07:01<1:07:22,  1.66it/s]




  9%|▉         | 690/7392 [07:02<1:06:46,  1.67it/s]




  9%|▉         | 691/7392 [07:02<1:07:17,  1.66it/s]




  9%|▉         | 692/7392 [07:03<1:07:31,  1.65it/s]




  9%|▉         | 693/7392 [07:04<1:07:08,  1.66it/s]




  9%|▉         | 694/7392 [07:04<1:07:17,  1.66it/s]




  9%|▉         | 695/7392 [07:05<1:06:51,  1.67it/s]




  9%|▉         | 696/7392 [07:05<1:06:36,  1.68it/s]




  9%|▉         | 697/7392 [07:0

 12%|█▏        | 906/7392 [09:13<1:05:47,  1.64it/s]




 12%|█▏        | 907/7392 [09:14<1:05:53,  1.64it/s]




 12%|█▏        | 908/7392 [09:15<1:05:48,  1.64it/s]




 12%|█▏        | 909/7392 [09:15<1:05:30,  1.65it/s]




 12%|█▏        | 910/7392 [09:16<1:09:01,  1.57it/s]




 12%|█▏        | 911/7392 [09:17<1:07:36,  1.60it/s]




 12%|█▏        | 912/7392 [09:17<1:15:08,  1.44it/s]




 12%|█▏        | 913/7392 [09:18<1:12:58,  1.48it/s]




 12%|█▏        | 914/7392 [09:19<1:11:08,  1.52it/s]




 12%|█▏        | 915/7392 [09:19<1:10:18,  1.54it/s]




 12%|█▏        | 916/7392 [09:20<1:08:53,  1.57it/s]




 12%|█▏        | 917/7392 [09:21<1:07:40,  1.59it/s]




 12%|█▏        | 918/7392 [09:21<1:06:51,  1.61it/s]




 12%|█▏        | 919/7392 [09:22<1:06:42,  1.62it/s]




 12%|█▏        | 920/7392 [09:22<1:05:59,  1.63it/s]




 12%|█▏        | 921/7392 [09:23<1:05:45,  1.64it/s]




 12%|█▏        | 922/7392 [09:24<1:05:34,  1.64it/s]




 12%|█▏        | 923/7392 [09:2

...

 85%|████████▍ | 6275/7392 [1:06:51<12:26,  1.50it/s]




 85%|████████▍ | 6276/7392 [1:06:52<12:25,  1.50it/s]




 85%|████████▍ | 6277/7392 [1:06:53<12:24,  1.50it/s]




 85%|████████▍ | 6278/7392 [1:06:53<12:25,  1.49it/s]




 85%|████████▍ | 6279/7392 [1:06:54<12:25,  1.49it/s]




 85%|████████▍ | 6280/7392 [1:06:55<12:22,  1.50it/s]




 85%|████████▍ | 6281/7392 [1:06:55<12:18,  1.50it/s]




 85%|████████▍ | 6282/7392 [1:06:56<12:26,  1.49it/s]




 85%|████████▍ | 6283/7392 [1:06:57<12:21,  1.49it/s]




 85%|████████▌ | 6284/7392 [1:06:57<12:20,  1.50it/s]




 85%|████████▌ | 6285/7392 [1:06:58<12:23,  1.49it/s]




 85%|████████▌ | 6286/7392 [1:06:59<12:22,  1.49it/s]




 85%|████████▌ | 6287/7392 [1:06:59<12:19,  1.49it/s]




 85%|████████▌ | 6288/7392 [1:07:00<12:36,  1.46it/s]




 85%|████████▌ | 6289/7392 [1:07:01<12:28,  1.47it/s]




 85%|████████▌ | 6290/7392 [1:07:02<12:24,  1.48it/s]




 85%|████████▌ | 6291/7392 [1:07:02<12:22,  1.48it/s]




 85%|████████▌

 88%|████████▊ | 6499/7392 [1:09:24<09:48,  1.52it/s]




 88%|████████▊ | 6500/7392 [1:09:24<09:57,  1.49it/s]




 88%|████████▊ | 6501/7392 [1:09:25<10:02,  1.48it/s]




 88%|████████▊ | 6502/7392 [1:09:26<09:51,  1.50it/s]




 88%|████████▊ | 6503/7392 [1:09:27<10:28,  1.41it/s]




 88%|████████▊ | 6504/7392 [1:09:27<10:26,  1.42it/s]




 88%|████████▊ | 6505/7392 [1:09:28<10:15,  1.44it/s]




 88%|████████▊ | 6506/7392 [1:09:29<09:58,  1.48it/s]




 88%|████████▊ | 6507/7392 [1:09:29<09:52,  1.49it/s]




 88%|████████▊ | 6508/7392 [1:09:30<09:52,  1.49it/s]




 88%|████████▊ | 6509/7392 [1:09:30<09:44,  1.51it/s]




 88%|████████▊ | 6510/7392 [1:09:31<09:38,  1.52it/s]




 88%|████████▊ | 6511/7392 [1:09:32<09:38,  1.52it/s]




 88%|████████▊ | 6512/7392 [1:09:32<09:32,  1.54it/s]




 88%|████████▊ | 6513/7392 [1:09:33<09:37,  1.52it/s]




 88%|████████▊ | 6514/7392 [1:09:34<09:32,  1.53it/s]




 88%|████████▊ | 6515/7392 [1:09:34<09:44,  1.50it/s]




 88%|████████▊

 91%|█████████ | 6723/7392 [1:11:55<08:03,  1.38it/s]




 91%|█████████ | 6724/7392 [1:11:55<07:54,  1.41it/s]




 91%|█████████ | 6725/7392 [1:11:56<07:50,  1.42it/s]




 91%|█████████ | 6726/7392 [1:11:57<07:45,  1.43it/s]




 91%|█████████ | 6727/7392 [1:11:57<07:41,  1.44it/s]




 91%|█████████ | 6728/7392 [1:11:58<07:35,  1.46it/s]




 91%|█████████ | 6729/7392 [1:11:59<09:12,  1.20it/s]




 91%|█████████ | 6730/7392 [1:12:00<08:44,  1.26it/s]




 91%|█████████ | 6731/7392 [1:12:00<08:18,  1.33it/s]




 91%|█████████ | 6732/7392 [1:12:01<08:00,  1.37it/s]




 91%|█████████ | 6733/7392 [1:12:02<07:48,  1.41it/s]




 91%|█████████ | 6734/7392 [1:12:03<07:51,  1.40it/s]




 91%|█████████ | 6735/7392 [1:12:03<07:44,  1.41it/s]




 91%|█████████ | 6736/7392 [1:12:04<07:36,  1.44it/s]




 91%|█████████ | 6737/7392 [1:12:05<07:32,  1.45it/s]




 91%|█████████ | 6738/7392 [1:12:05<07:32,  1.45it/s]




 91%|█████████ | 6739/7392 [1:12:06<07:33,  1.44it/s]




 91%|█████████

 94%|█████████▍| 6947/7392 [1:14:28<05:05,  1.46it/s]




 94%|█████████▍| 6948/7392 [1:14:29<05:07,  1.45it/s]




 94%|█████████▍| 6949/7392 [1:14:30<05:12,  1.42it/s]




 94%|█████████▍| 6950/7392 [1:14:30<05:08,  1.43it/s]




 94%|█████████▍| 6951/7392 [1:14:31<05:05,  1.45it/s]




 94%|█████████▍| 6952/7392 [1:14:32<04:59,  1.47it/s]




 94%|█████████▍| 6953/7392 [1:14:32<04:56,  1.48it/s]




 94%|█████████▍| 6954/7392 [1:14:33<04:53,  1.49it/s]




 94%|█████████▍| 6955/7392 [1:14:34<04:50,  1.51it/s]




 94%|█████████▍| 6956/7392 [1:14:34<04:47,  1.52it/s]




 94%|█████████▍| 6957/7392 [1:14:35<04:45,  1.52it/s]




 94%|█████████▍| 6958/7392 [1:14:36<04:43,  1.53it/s]




 94%|█████████▍| 6959/7392 [1:14:36<04:42,  1.53it/s]




 94%|█████████▍| 6960/7392 [1:14:37<04:41,  1.53it/s]




 94%|█████████▍| 6961/7392 [1:14:38<04:43,  1.52it/s]




 94%|█████████▍| 6962/7392 [1:14:38<04:46,  1.50it/s]




 94%|█████████▍| 6963/7392 [1:14:39<04:44,  1.51it/s]




 94%|█████████

 97%|█████████▋| 7171/7392 [1:16:56<02:26,  1.51it/s]




 97%|█████████▋| 7172/7392 [1:16:57<02:26,  1.50it/s]




 97%|█████████▋| 7173/7392 [1:16:58<02:28,  1.48it/s]




 97%|█████████▋| 7174/7392 [1:16:59<02:26,  1.49it/s]




 97%|█████████▋| 7175/7392 [1:16:59<02:25,  1.49it/s]




 97%|█████████▋| 7176/7392 [1:17:00<02:23,  1.50it/s]




 97%|█████████▋| 7177/7392 [1:17:01<02:22,  1.51it/s]




 97%|█████████▋| 7178/7392 [1:17:01<02:21,  1.51it/s]




 97%|█████████▋| 7179/7392 [1:17:02<02:19,  1.52it/s]




 97%|█████████▋| 7180/7392 [1:17:02<02:18,  1.53it/s]




 97%|█████████▋| 7181/7392 [1:17:03<02:19,  1.51it/s]




 97%|█████████▋| 7182/7392 [1:17:04<02:17,  1.52it/s]




 97%|█████████▋| 7183/7392 [1:17:04<02:19,  1.50it/s]




 97%|█████████▋| 7184/7392 [1:17:05<02:17,  1.51it/s]




 97%|█████████▋| 7185/7392 [1:17:06<02:16,  1.51it/s]




 97%|█████████▋| 7186/7392 [1:17:06<02:16,  1.51it/s]




 97%|█████████▋| 7187/7392 [1:17:07<02:15,  1.51it/s]




 97%|█████████

In [101]:
rf_results_df = rf_results_df.merge(rf_lags_df, on='Timestamp')
rf_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags,TotEntr_PUF_allcols,TotEntr_PUF_lags
Timestamp,,,,,
2018-09-24 00:00:00,20,15.470000,15.475000,15.4700,15.475000
2018-09-24 00:30:00,0,0.772719,0.703395,55.3450,0.703395
2018-09-24 01:00:00,0,0.000000,0.000000,38.8900,0.000000
2018-09-24 01:30:00,0,0.000000,0.000000,32.0025,0.000000
2018-09-24 02:00:00,0,0.000000,0.000000,15.5425,0.000000


In [102]:
rf_runtime_df = pd.concat([rf_runtime_df, rf_runtime_PUF_lags], axis = 1)
rf_runtime_df.head()

,PKF_allcols_rf_runtime,PKF_lags_rf_runtime,PUF_allcols_rf_runtime,PUF_lags_rf_runtime
0,371.837343,386.499835,5577.384185,4768.740998


In [105]:
rf_results_df.to_csv('./../../../Databases/rf_results.csv')
rf_runtime_df.to_csv('./../../../Databases/rf_runtime_scenarios.csv', index=False)

### 10 days: All columns, unknown future

Creating lags on the fly from new predictions and refitting the model in each iteration.

In [36]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

rf_X_train = pd.concat([X_train, X_dev])
rf_X_train.sort_index(inplace=True)
rf_y_train = pd.concat([y_train, y_dev])
rf_y_train.sort_index(inplace=True)

In [37]:
rf_PUF_all10_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, random_state=42)
rf_PUF_all10_model.fit(rf_X_train, rf_y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

A per previous tests run, each prediction takes an average of 5.4 minutes to be calculated using the predefined custom function. That means that for all rows on X_test (7392) it would be needed a complete month running non-stop in order to estimate all the predictions.

Therefore, the predictions for the first 10 days (480 observations) will be computed.

In [17]:
start_PUF10_allcols = time.time()

rf_all10_df = get_future_preds(no_lags_marag, rf_X_train, X_test[:480], rf_PUF_all10_model)

end_PUF10_allcols = time.time()
rf_runtime_PUF10_allcols = end_PUF10_allcols - start_PUF10_allcols
rf_runtime_PUF10_allcols = pd.DataFrame(pd.Series(rf_runtime_PUF10_allcols, name = 'PUF_allcols_rf_runtime_10d'))
rf_all10_df.columns = ['TotEntr_PUF_allcols_10d']

In [31]:
rf_observed_values = pd.DataFrame(y_test)
rf_observed_values.columns = ['Observed_values']
rf_10_df = rf_observed_values[:480].merge(pd.DataFrame(rf_all_df), how='left', on='Timestamp')
rf_10_df.head()

,Observed_values,TotEntr_PUF_allcols_10d
Timestamp,,
2018-09-24 00:00:00,20,15.470000
2018-09-24 00:30:00,0,5.364594
2018-09-24 01:00:00,0,0.396983
2018-09-24 01:30:00,0,0.000000
2018-09-24 02:00:00,0,0.000000


In [34]:
rf_runtime10 = rf_runtime_PUF10_allcols
rf_runtime10.head()

,PUF_allcols_lr_runtime_10d
0,181398.481392


### 10 days: All columns, just lags

In [15]:
lags_PUF_df = marag_df.drop(['Temperature', 'Precipitation', 'Open/Closed',
       'Weekday/Weekend'], axis = 1)

X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(lags_PUF_df.drop(['TotalEntries'], axis=1), lags_PUF_df['TotalEntries'])

rf_lags_X_train = pd.concat([X_train, X_dev])
rf_lags_X_train.sort_index(inplace=True)
rf_lags_y_train = pd.concat([y_train, y_dev])
rf_lags_y_train.sort_index(inplace=True)

In [23]:
rf_PUF_lags10_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, random_state=42)
rf_PUF_lags10_model.fit(rf_lags_X_train, rf_lags_y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [24]:
start_PUF10_lags = time.time()

rf_lags10_df = get_future_preds(no_lags_marag, rf_lags_X_train, X_test[:480], rf_PUF_lags10_model, justlags=True)

end_PUF10_lags = time.time()
rf_runtime_PUF10_lags = end_PUF10_lags - start_PUF10_lags
rf_runtime_PUF10_lags = pd.DataFrame(pd.Series(rf_runtime_PUF10_lags, name = 'PUF_lags_rf_runtime_10d'))
rf_lags10_df.columns = ['TotEntr_PUF_lags_10d']


 57%|█████▋    | 275/480 [28:50:23<21:38:46, 380.13s/it]


100%|██████████| 480/480 [50:35:58<00:00, 382.46s/it]

In [33]:
rf_10_df = rf_10_df.merge(rf_lags_df, on='Timestamp')
rf_10_df.head()

,Observed_values,TotEntr_PUF_allcols_10d,TotEntr_PUF_lags_10d
Timestamp,,,
2018-09-24 00:00:00,20,15.470000,15.475000
2018-09-24 00:30:00,0,5.364594,0.703488
2018-09-24 01:00:00,0,0.396983,0.000000
2018-09-24 01:30:00,0,0.000000,0.000000
2018-09-24 02:00:00,0,0.000000,0.000000


In [35]:
rf_runtime10 = pd.concat([rf_runtime10, rf_runtime_PUF10_lags], axis = 1)
rf_runtime10.head()

,PUF_allcols_lr_runtime_10d,PUF_lags_rf_runtime_10d
0,181398.481392,182158.419702


In [ ]:
rf_10_df.to_csv('./../../../Databases/10day_results.csv')
rf_runtime10.to_csv('./../../../Databases/10day_runtime_scenarios.csv', index=False)

## Daily data (1 obs. / day)

### Model tuning

In [169]:
X_train, X_dev, rf_dX_test, y_train, y_dev, rf_dy_test = train_dev_test_split(marag_daily.drop(['TotalEntries'], axis=1), 
                                                                      marag_daily['TotalEntries'])

rf_dX_train = pd.concat([X_train, X_dev])
rf_dX_train.sort_index(inplace=True)
rf_dy_train = pd.concat([y_train, y_dev])
rf_dy_train.sort_index(inplace=True)

In [170]:
# RandomForest day baseline

rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)
RF_dbaseline_y_hat = rf.predict(X_dev)

/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [171]:
baseline_d_RMSE = pd.DataFrame(pd.Series(np.sqrt(mean_squared_error(y_dev, RF_dbaseline_y_hat)), name = 'RMSE'))
baseline_d_RMSE

,RMSE
0,950.783378


In [107]:
n_estimators = [100, 200, 400, 600]
max_depth = [10, 30, 60, 100]
best_d_RMSE = []

start_drf_tune = time.time()

for ne in tqdm(n_estimators):
    for md in tqdm(max_depth):
        rf = RandomForestRegressor(n_estimators = ne, max_depth = md, n_jobs = -1, random_state = 42)
        rf.fit(X_train, y_train)
        y_dev_hat = rf.predict(X_dev)
        RMSE_rfparams = sqrt(mean_squared_error(y_dev, y_dev_hat))
        
        if pd.DataFrame(best_d_RMSE).empty:
            best_d_RMSE = [ne, md, RMSE_rfparams]
        else:
            if RMSE_rfparams < best_d_RMSE[2]:
                best_d_RMSE = [ne, md, RMSE_rfparams]
        
end_drf_tune = time.time()
drf_tune_runtime = end_drf_tune - start_drf_tune






  0%|          | 0/4 [00:00<?, ?it/s]





  0%|          | 0/4 [00:00<?, ?it/s]





 25%|██▌       | 1/4 [00:00<00:00,  3.68it/s]





 50%|█████     | 2/4 [00:00<00:00,  3.67it/s]





 75%|███████▌  | 3/4 [00:00<00:00,  3.62it/s]





100%|██████████| 4/4 [00:01<00:00,  3.63it/s]




 25%|██▌       | 1/4 [00:01<00:03,  1.11s/it]





  0%|          | 0/4 [00:00<?, ?it/s]





 25%|██▌       | 1/4 [00:00<00:01,  2.42it/s]





 50%|█████     | 2/4 [00:00<00:00,  2.25it/s]





 75%|███████▌  | 3/4 [00:01<00:00,  2.17it/s]





100%|██████████| 4/4 [00:01<00:00,  2.25it/s]




 50%|█████     | 2/4 [00:02<00:02,  1.33s/it]





  0%|          | 0/4 [00:00<?, ?it/s]





 25%|██▌       | 1/4 [00:00<00:01,  1.72it/s]





 50%|█████     | 2/4 [00:01<00:01,  1.55it/s]





 75%|███████▌  | 3/4 [00:02<00:00,  1.46it/s]





100%|██████████| 4/4 [00:03<00:00,  1.34it/s]




 75%|███████▌  | 3/4 [00:06<00:01,  1.85s/it]





  0%|          | 0/4 [00:00<?, ?it/s]





 25%|██▌       | 1

In [109]:
print(best_d_RMSE, drf_tune_runtime)

[200, 10, 857.2220981393417] 10.233720064163208


### All columns, known future

In [111]:
start_dPKF_allcols = time.time()

rf_PKF_d = RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1, verbose=10, random_state=42)
rf_PKF_d.fit(rf_dX_train, rf_dy_train)
rf_dy_hat = rf_PKF_d.predict(rf_dX_test)
rf_dy_hat = pd.Series(rf_dy_hat, index=rf_dy_test.index, name='day_PKF_allcols')

end_dPKF_allcols = time.time()
rf_runtime_dPKF_allcols = end_dPKF_allcols - start_dPKF_allcols
rf_runtime_dPKF_allcols = pd.DataFrame(pd.Series(rf_runtime_dPKF_allcols, name = 'day_PKF_allcols_rf_runtime'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapse

building tree 1 of 200building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200

building tree 7 of 200
building tree 8 of 200
building tree 9 of 200building tree 10 of 200
building tree 11 of 200

building tree 12 of 200building tree 13 of 200

building tree 14 of 200building tree 15 of 200
building tree 16 of 200
building tree 17 of 200building tree 18 of 200


building tree 19 of 200
building tree 20 of 200building tree 21 of 200

building tree 22 of 200building tree 23 of 200building tree 24 of 200building tree 25 of 200

building tree 26 of 200
building tree 27 of 200


building tree 28 of 200
building tree 29 of 200
building tree 30 of 200building tree 31 of 200

building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200bu

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Para

In [112]:
rf_d_observed_values = pd.DataFrame(rf_dy_test)
rf_d_observed_values.columns = ['daily_observed_values']
rf_dailyresults_df = rf_d_observed_values.merge(pd.DataFrame(rf_dy_hat), how='left', on='Timestamp')
rf_dailyresults_df.head()

,daily_observed_values,day_PKF_allcols
Timestamp,,
2018-09-24,5719,9665.562973
2018-09-25,9998,9396.381812
2018-09-26,9816,9674.329971
2018-09-27,10087,9779.599758
2018-09-28,10742,10509.663615


In [113]:
rf_runtime_dailydf = rf_runtime_dPKF_allcols
rf_runtime_dailydf.head()

,day_PKF_allcols_rf_runtime
0,0.414059


### Just lags, known future

In [125]:
lags_dPKF_df = marag_daily.drop(['T', 'PPT', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, rf_dlags_X_test, y_train, y_dev, rf_dlags_y_test = train_dev_test_split(lags_dPKF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_dPKF_df['TotalEntries'])

rf_dlags_X_train = pd.concat([X_train, X_dev])
rf_dlags_X_train.sort_index(inplace=True)
rf_dlags_y_train = pd.concat([y_train, y_dev])
rf_dlags_y_train.sort_index(inplace=True)

In [127]:
start_dPKF_lags = time.time()
drf_lags_model = RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1, random_state=42) 
drf_lags_model.fit(rf_dlags_X_train, rf_dlags_y_train)
rf_dlags_y_hat = drf_lags_model.predict(rf_dlags_X_test)
rf_dlags_y_hat = pd.Series(rf_dlags_y_hat, index=rf_dlags_y_test.index, name = 'day_PKF_lags')

end_dPKF_lags = time.time()
rf_runtime_dPKF_lags = end_dPKF_lags - start_dPKF_lags
rf_runtime_dPKF_lags = pd.DataFrame(pd.Series(rf_runtime_dPKF_lags, name = 'day_PKF_lags_rf_runtime'))

In [128]:
rf_dailyresults_df = rf_dailyresults_df.merge(pd.DataFrame(rf_dlags_y_hat), how='left', on='Timestamp')
rf_dailyresults_df.head()

,daily_observed_values,day_PKF_allcols,day_PKF_lags
Timestamp,,,
2018-09-24,5719,9665.562973,9553.809870
2018-09-25,9998,9396.381812,9229.229885
2018-09-26,9816,9674.329971,9789.210041
2018-09-27,10087,9779.599758,9838.006070
2018-09-28,10742,10509.663615,10486.754899


In [129]:
rf_runtime_dailydf = pd.concat([rf_runtime_dailydf, rf_runtime_dPKF_lags], axis=1)
rf_runtime_dailydf.head()

,day_PKF_allcols_rf_runtime,day_PKF_lags_rf_runtime
0,0.414059,0.403259


When working with daily frequency, the refitting process of the custom function doesn't take much time, so we will proceed extracting the predictions as usual.

### All columns, unknown future

In [148]:
X_train, X_dev, rf_dX_test, y_train, y_dev, rf_dy_test = train_dev_test_split(marag_daily.drop(['TotalEntries'], axis=1), 
                                                                      marag_daily['TotalEntries'])

rf_dX_train = pd.concat([X_train, X_dev])
rf_dX_train.sort_index(inplace=True)
rf_dy_train = pd.concat([y_train, y_dev])
rf_dy_train.sort_index(inplace=True)

In [149]:
drf_PUF_allcols_model = RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1, random_state=42)
drf_PUF_allcols_model.fit(rf_dX_train, rf_dy_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [150]:
start_dPUF_allcols = time.time()

drf_allcols_df = get_future_preds(daily_nolags_marag, rf_dX_train, rf_dX_test, drf_PUF_allcols_model, daily=True)

end_dPUF_allcols = time.time()
drf_runtime_PUF_allcols = end_dPUF_allcols - start_dPUF_allcols
drf_runtime_PUF_allcols = pd.DataFrame(pd.Series(drf_runtime_PUF_allcols, name = 'day_PUF_allcols_rf_runtime'))
drf_allcols_df.columns = ['day_PUF_allcols']






  0%|          | 0/154 [00:00<?, ?it/s]




  1%|          | 1/154 [00:00<00:46,  3.32it/s]




  1%|▏         | 2/154 [00:00<00:51,  2.97it/s]




  2%|▏         | 3/154 [00:01<01:04,  2.35it/s]




  3%|▎         | 4/154 [00:01<01:05,  2.30it/s]




  3%|▎         | 5/154 [00:02<01:04,  2.31it/s]




  4%|▍         | 6/154 [00:02<01:02,  2.35it/s]




  5%|▍         | 7/154 [00:03<01:02,  2.35it/s]




  5%|▌         | 8/154 [00:03<01:05,  2.22it/s]




  6%|▌         | 9/154 [00:04<01:09,  2.09it/s]




  6%|▋         | 10/154 [00:04<01:06,  2.16it/s]




  7%|▋         | 11/154 [00:04<01:03,  2.24it/s]




  8%|▊         | 12/154 [00:05<01:02,  2.26it/s]




  8%|▊         | 13/154 [00:05<01:01,  2.31it/s]




  9%|▉         | 14/154 [00:06<01:00,  2.32it/s]




 10%|▉         | 15/154 [00:06<00:59,  2.35it/s]




 10%|█         | 16/154 [00:07<00:59,  2.33it/s]




 11%|█         | 17/154 [00:07<00:58,  2.34it/s]




 12%|█▏        | 18/154 [00:07<00:57,  2.36it/s]




 12%|█

In [154]:
rf_dailyresults_df = rf_dailyresults_df.merge(pd.DataFrame(drf_allcols_df), how='left', on='Timestamp')
rf_dailyresults_df.head()

,daily_observed_values,day_PKF_allcols,day_PKF_lags,day_PUF_allcols
Timestamp,,,,
2018-09-24,5719,9665.562973,9553.809870,9665.562973
2018-09-25,9998,9396.381812,9229.229885,9727.018141
2018-09-26,9816,9674.329971,9789.210041,9887.010106
2018-09-27,10087,9779.599758,9838.006070,9983.932062
2018-09-28,10742,10509.663615,10486.754899,10670.555464


In [158]:
rf_runtime_dailydf = pd.concat([rf_runtime_dailydf, drf_runtime_PUF_allcols], axis=1)
rf_runtime_dailydf.head()

,day_PKF_allcols_rf_runtime,day_PKF_lags_rf_runtime,day_PUF_allcols_rf_runtime
0,0.414059,0.403259,67.779633


### Just lags, unknown future

In [159]:
lags_dPKF_df = marag_daily.drop(['T', 'PPT', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, rf_dlags_X_test, y_train, y_dev, rf_dlags_y_test = train_dev_test_split(lags_dPKF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_dPKF_df['TotalEntries'])

rf_dlags_X_train = pd.concat([X_train, X_dev])
rf_dlags_X_train.sort_index(inplace=True)
rf_dlags_y_train = pd.concat([y_train, y_dev])
rf_dlags_y_train.sort_index(inplace=True)

In [160]:
drf_PUF_lags_model = RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1, random_state=42)
drf_PUF_lags_model.fit(rf_dlags_X_train, rf_dlags_y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [161]:
start_dPUF_lags = time.time()

drf_lags_df = get_future_preds(daily_nolags_marag, rf_dlags_X_train, rf_dlags_X_test, drf_PUF_lags_model, daily=True, justlags=True)

end_dPUF_lags = time.time()
drf_runtime_PUF_lags = end_dPUF_lags - start_dPUF_lags
drf_runtime_PUF_lags = pd.DataFrame(pd.Series(drf_runtime_PUF_lags, name = 'day_PUF_lags_rf_runtime'))
drf_lags_df.columns = ['day_PUF_lags']






  0%|          | 0/154 [00:00<?, ?it/s]




  1%|          | 1/154 [00:00<00:51,  2.95it/s]




  1%|▏         | 2/154 [00:00<00:55,  2.74it/s]




  2%|▏         | 3/154 [00:01<01:01,  2.44it/s]




  3%|▎         | 4/154 [00:01<01:02,  2.41it/s]




  3%|▎         | 5/154 [00:02<01:01,  2.41it/s]




  4%|▍         | 6/154 [00:02<01:06,  2.23it/s]




  5%|▍         | 7/154 [00:03<01:04,  2.28it/s]




  5%|▌         | 8/154 [00:03<01:03,  2.31it/s]




  6%|▌         | 9/154 [00:03<01:06,  2.18it/s]




  6%|▋         | 10/154 [00:04<01:04,  2.25it/s]




  7%|▋         | 11/154 [00:04<01:02,  2.28it/s]




  8%|▊         | 12/154 [00:05<01:00,  2.33it/s]




  8%|▊         | 13/154 [00:05<01:00,  2.34it/s]




  9%|▉         | 14/154 [00:06<01:03,  2.22it/s]




 10%|▉         | 15/154 [00:06<01:01,  2.25it/s]




 10%|█         | 16/154 [00:07<00:59,  2.31it/s]




 11%|█         | 17/154 [00:07<00:58,  2.34it/s]




 12%|█▏        | 18/154 [00:07<00:58,  2.33it/s]




 12%|█

In [162]:
rf_dailyresults_df = rf_dailyresults_df.merge(pd.DataFrame(drf_lags_df), how='left', on='Timestamp')
rf_dailyresults_df.head()

,daily_observed_values,day_PKF_allcols,day_PKF_lags,day_PUF_allcols,day_PUF_lags
Timestamp,,,,,
2018-09-24,5719,9665.562973,9553.809870,9665.562973,9553.809870
2018-09-25,9998,9396.381812,9229.229885,9727.018141,9512.557773
2018-09-26,9816,9674.329971,9789.210041,9887.010106,9783.892463
2018-09-27,10087,9779.599758,9838.006070,9983.932062,9888.053528
2018-09-28,10742,10509.663615,10486.754899,10670.555464,10623.208573


In [163]:
rf_runtime_dailydf = pd.concat([rf_runtime_dailydf, drf_runtime_PUF_lags], axis=1)
rf_runtime_dailydf.head()

,day_PKF_allcols_rf_runtime,day_PKF_lags_rf_runtime,day_PUF_allcols_rf_runtime,day_PUF_lags_rf_runtime
0,0.414059,0.403259,67.779633,67.729317


In [164]:
rf_dailyresults_df.to_csv('./../../../Databases/rf_dailyresults.csv')
rf_runtime_dailydf.to_csv('./../../../Databases/rf_runtime_dailyscenarios.csv', index=False)